## Group 15 - Contraceptive method used among Indonesian women

Kenna Roozendaal, Joy Cai, Naura Ambareswari

#### Introduction

Indonesia experienced a notable decline in its fertility rates between 1984 and 1987. The increased use of contraceptives is a significant factor in the drop in fertility due to more women gaining access to and awareness of available family planning resources compared to previous years. To better understand and address the declining fertility trend, the National Indonesia Contraceptive Prevalence Survey conducted a survey on family planning as well as collected women’s sociodemographic background.

The data taken from the survey will be used to answer the question *“Given the wife's age, religion, and standard of living, what is the preferred contraceptive method used?”*. We will be using the Contraceptive Method Choice Data Set that sampled married women in Indonesia who were not pregnant or were unaware they were pregnant (Lim, 1997). The goal was to predict women’s contraceptive method of choice (no use, long-term methods, short-term methods) based on characteristics like age, religion, standard of living, education, etc.

Link to dataset: https://archive.ics.uci.edu/ml/datasets/Contraceptive+Method+Choice 

#### Methods

First, the dataset will need to be filtered in order to only get the columns that are relvant to answering the question. Since there are no headers provided with the dataset, they will need to be manually set. The columns used from the dataset are wife’s age, religion, education, standard of living, and contraceptive method used. Religion was chosen as a predictor because it influences the type of contraceptive used and Indonesia holds the largest Muslim population (Hefner, 2020; Hill et al., 2014). Education and age are predictors because age, socioeconomic level, and level of education, particularly educational lag, have been found to impact women’s contraceptive methods (Sámano et al., 2019). 

RELEVANT COLUMNS:
- Wife's age
- Wife's education level (1=low 2,3,4=high)
- Wife's religion (0=not Islam 1=Islam)
- Standard of living (1=low 2,3,4=high)
- Contraceptive method used (1= no use 2=long-term 3=short-term)

Aftering filtering the dataset, will be read into Python and the data will be split into a training and testing set, 75% for the training set and 25% for the testing set. This will allow us to evaluate the accuracy of our model for predicting the prefered method of contraception. To create the most accurate predictor, the model will be tuned by testing different k-values for k-nearest neighbors. As such, a standard <code>5-fold-cross-validation</code> will be conducted using the GridSearchCV method from the scikit-learn package. Using the best value for k, the model with maximized efficiency can be used.

Using the tuned model, we will be able to predict the method of contraception used given information about the wife using the <code>predict</code> method. 


#### Expected outcomes and significance

Women with a higher standard of living and higher education level tend to use contraceptives longer than other groups because they are more aware of the available family planning resources and other resources beyond what the government offers. Alternately, women with lower education levels and a lower standard of living are more likely to use contraceptives for a shorter period or not use them at all (Iseyemi et al., 2017). 

Predicting women’s contraceptive choice can help healthcare providers and policymakers ensure proper access to their preferred contraception. Identifying sociodemographic groups that use or do not use contraceptives can assess disparities in access, utilization, and education of contraception. Despite Indonesia’s decline in fertility, contraceptive use can lower rates of unplanned pregnancies as well as maternal and infant mortality. 
This research can further inquire on the accessibility of contraception and family planning for all sociodemographic groups in Indonesia, and how it can be improved in order to ensure equality and promote autonomy and control over women’s reproductive health. 


#### Preliminary exploratory data analysis

We first want to load the csv file into a dataframe so that we can better visualize the data. Appropriate headers are added and only the columns that have variables that are not related to the question are removed.

In [9]:
import altair as alt
import numpy as np
import pandas as pd
import sklearn
from sklearn.compose import make_column_transformer
from sklearn.metrics import confusion_matrix
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_validate,
    train_test_split,
)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

alt.data_transformers.disable_max_rows()
alt.renderers.enable("mimetype")

RendererRegistry.enable('mimetype')

In [2]:
cmc_dataset = pd.read_csv("data/cmc.data.csv", header=None,
                     names=[
                         "wife_age", 
                         "wife_education", #1=low 2,3,4=high
                         "husband_education", #1=low 2,3,4=high
                         "num_children_born",
                         "wife_religion", #0=not Islam 1=Islam
                         "wife_working", #0=yes 1=no
                         "husband_occupation", #1,2,3,4 (categorical)
                         "standard_of_living_index", #1=low 2,3,4=high
                         "media_exposure", #0=good 1=not good
                         "contraceptive_method_used", #1= no use 2=long-term 3=short-term
                     ]
                     )
cmc_dataset

,wife_age,wife_education,husband_education,num_children_born,wife_religion,wife_working,husband_occupation,standard_of_living_index,media_exposure,contraceptive_method_used
0,24,2,3,3,1,1,2,3,0,1
1,45,1,3,10,1,1,3,4,0,1
2,43,2,3,7,1,1,3,4,0,1
3,42,3,2,9,1,1,3,3,0,1
4,36,3,3,8,1,1,3,2,0,1
...,...,...,...,...,...,...,...,...,...,...
1468,33,4,4,2,1,0,2,4,0,3
1469,33,4,4,3,1,1,1,4,0,3
1470,39,3,3,8,1,0,1,4,0,3
1471,33,3,3,4,1,0,2,2,0,3


In [3]:
cmc_dataset_filtered = cmc_dataset.loc[:, ["wife_age", "wife_education","wife_religion","standard_of_living_index", "contraceptive_method_used"]]
cmc_dataset_filtered

,wife_age,wife_education,wife_religion,standard_of_living_index,contraceptive_method_used
0,24,2,1,3,1
1,45,1,1,4,1
2,43,2,1,4,1
3,42,3,1,3,1
4,36,3,1,2,1
...,...,...,...,...,...
1468,33,4,1,4,3
1469,33,4,1,4,3
1470,39,3,1,4,3
1471,33,3,1,2,3


Now we want to split our data into a training and testing set. The preliminary data analysis will be an exploratory analysis that uses data from the training set only. In order to reduce human bias, only the training set will be visualized. For this project, the dataset will be split 75% training and 25% testing. 

In [10]:
cmc_train, cmc_test = train_test_split(cmc_dataset_filtered, test_size=0.25, random_state=123)
cmc_train.head()

,wife_age,wife_education,wife_religion,standard_of_living_index,contraceptive_method_used
1208,40,1,1,2,1
1297,35,4,1,4,2
736,19,3,1,3,3
186,20,2,1,2,1
1027,41,2,0,2,1


In our preliminary analysis, we want to be able to summarize useful information about the training set in a table. As such, we want to be able to see if there is any missing data, and the means and medians of our predictor variables. 

In [11]:
missing_data = cmc_train.isna().sum()
missing_data

wife_age                     0
wife_education               0
wife_religion                0
standard_of_living_index     0
contraceptive_method_used    0
dtype: int64

There is no missing data in our dataset, thus we do not need to filter out any incomplete rows.

In [6]:
variable_agg = cmc_train.agg(["mean", "median", "max", "min"])
variable_agg

,wife_age,wife_education,wife_religion,standard_of_living_index,contraceptive_method_used
mean,32.570652,2.950181,0.856884,3.130435,1.906703
median,32.000000,3.000000,1.000000,3.000000,2.000000
max,49.000000,4.000000,1.000000,4.000000,3.000000
min,16.000000,1.000000,0.000000,1.000000,1.000000


From the table above, we are able to obtain information about the distribution of the data in the training set. The mean and median gives an idea of the distribution of the data and which groups are dominant for each predictor. Using the min and max, we can confirm that observations spanning the entire range of the dataset are included. 

**Visualization**

We chose our predictors based on academic papers that have found a correlation between each variable and the method of contraception used. As such we want to be able to see if each variable demonstrates a clear correlation. 

**Wife's age**

In [17]:
wife_age_correlation = alt.Chart(cmc_train, title = "Age vs. Method of Contraception Used").mark_point().encode(
    x = alt.X("wife_age", title = "Wife's age (years)"),
    y = alt.Y("count()", title = "Number of records"),
).facet("contraceptive_method_used")
wife_age_correlation

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


There does not seem to be a clear correlation between age and no method of contraception used. However, for short-term and long-term contraception, we can clearly see a peak at around 35 and 30 years old respectively. 

**Wife's education**

In [19]:
wife_education_correlation = alt.Chart(cmc_train, title = "Distribution of Education For Each Method of Contraception Used").mark_bar(size=10).encode(
    x = alt.X("contraceptive_method_used", title = "Contraceptive method used"),
    y = alt.Y("count()", title = "Number of records"),
    color = alt.Color("wife_education",legend=alt.Legend(title="Education level"))
).properties(width=100)
wife_education_correlation

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


From the graph, we can see that as the level of education increases, there is a greater proportion of of women who chose to use contraceptives. 

**Standard of living**

In [20]:
living_standard_correlation = alt.Chart(cmc_train, title = "Distribution of Standard of Living For Each Method of Contraception Used").mark_bar(size=10).encode(
    x = alt.X("contraceptive_method_used", title = "Contraceptive method used"),
    y = alt.Y("count()", title = "Number of records"),
    color = alt.Color("standard_of_living_index", legend=alt.Legend(title="Standard of living index"))
).properties(width=100)
living_standard_correlation

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


For individuals who have a standard of living of 1-3, it can clearly be seen that the proportion that use contraceptions decreases significantly for short-term (2) and long-term (3) methods. In addition, a larger percentage of the bars for 2 and 3 are taken up by individuals with a very high standard of living (4), meaning that there is a greater proportion that use contraceptives. 

**Religion**

In [21]:
wife_religion_correlation = alt.Chart(cmc_train, title = "Distribution of Religion For Each Method of Contraception Used").mark_bar(size=10).encode(
    x = alt.X("contraceptive_method_used", title = "Contraceptive method used"),
    y = alt.Y("count()", title = "Number of records"),
    color = alt.Color("wife_religion", legend=alt.Legend(title="Religion"))
).properties(width=100)
wife_religion_correlation

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


From the graph, we can see that the greater number of women surveyed were religious and used no method of contraception. However, when focusing on the invididual bars, the distribution of religion and non-religious women for each method of contraception does not differ greatly. 

#### References

Hefner, R. W. (2020). Islam and covenantal pluralism in Indonesia: A critical juncture analysis. The Review of Faith & International Affairs, 18(2), 1-17.

Hill, N. J., Siwatu, M., & Robinson, A. K. (2014). “My religion picked my birth control”: The influence of religion on contraceptive use. Journal of religion and health, 53, 825-833.

Iseyemi, A., Zhao, Q., McNicholas, C., & Peipert, J. F. (2017). Socioeconomic status as a risk factor for unintended pregnancy in the contraceptive CHOICE project. Obstetrics and gynecology, 130(3), 609.

Lim, T. (1997). UCI Machine Learning Repository [http://archive.ics.uci.edu/ml]. Irvine, CA: University of California, School of Information and Computer Science.

Sámano, R., Martínez-Rojano, H., Chico-Barba, G., Sánchez-Jiménez, B., Sam-Soto, S., Rodríguez-Ventura, A. L., ... & Sclavo-Melo, S. (2019). Sociodemographic factors associated with the knowledge and use of birth control methods in adolescents before and after pregnancy. International journal of environmental research and public health, 16(6), 1022.
